In [2]:
from PIL import Image
import torch
import math 
import function.utils_rotate as utils_rotate
from IPython.display import display
import os
import function.helper as helper
import cv2
import torch
import numpy as np
from time import time

In [3]:
class LicensePlateTracker:
    def __init__(self):
        self.tracked_plates = []
        self.next_id = 0
        
    def update(self, detections):
        matched_tracks = set()
        
        # Process new detections
        for detection in detections:
            matched = False
            bbox = detection[:4]
            conf = detection[4]
            
            # Compare with existing tracks
            for track in self.tracked_plates:
                if self._iou(bbox, track['bbox']) > 0:  # IOU threshold
                    track['bbox'] = bbox
                    track['conf'] = conf
                    track['age'] = 0
                    matched_tracks.add(track['id'])
                    matched = True
                    break
            
            # Create new track if no match
            if not matched:
                self.tracked_plates.append({
                    'id': self.next_id,
                    'bbox': bbox,
                    'conf': conf,
                    'age': 0
                })
                matched_tracks.add(self.next_id)
                self.next_id += 1
        
        # Update age of unmatched tracks
        self.tracked_plates = [track for track in self.tracked_plates 
                             if track['id'] in matched_tracks or track['age'] < 5]
        
        # Increment age for unmatched tracks
        for track in self.tracked_plates:
            if track['id'] not in matched_tracks:
                track['age'] += 1
    
    def _iou(self, bbox1, bbox2):
        # Calculate intersection over union
        x1 = max(bbox1[0], bbox2[0])
        y1 = max(bbox1[1], bbox2[1])
        x2 = min(bbox1[2], bbox2[2])
        y2 = min(bbox1[3], bbox2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
        area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
        union = area1 + area2 - intersection
        
        return intersection / (union + 1e-6)

In [4]:
yolo_LP_detect = torch.hub.load('yolov5', 'custom', path='model/LP_detector.pt', force_reload=True, source='local')
# yolo_license_plate = torch.hub.load('yolov5', 'custom', path='model/LP_ocr.pt', force_reload=True, source='local')
yolo_LP_detect.conf = 0.1

YOLOv5  v6.1-179-gf3fecf9 torch 2.5.1 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

d:\work\AIVISION\license-plate\license-plate\license-plate\License-Plate-Recognition\yolov5\models\experimental.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open a

In [5]:
import cv2
import numpy as np


polygon_points = np.array([(89, 607), (331, 344), (1296, 352), (1313, 641)], np.int32)


cap = cv2.VideoCapture('./test_image/test3.mp4')  
tracker = LicensePlateTracker()  # Giả sử đây là tracker đã được định nghĩa

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_LP_detect(frame)
    detections = results.xyxy[0].cpu().numpy()


    filtered_detections = []
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])

        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Kiểm tra xem tâm có nằm trong đa giác không
        # Hàm pointPolygonTest trả về:
        #   > 0 nếu điểm nằm bên trong,
        #   = 0 nếu điểm nằm trên biên,
        #   < 0 nếu điểm nằm bên ngoài
        if cv2.pointPolygonTest(polygon_points, (center_x, center_y), False) >= 0:
            filtered_detections.append(det)

    tracker.update(filtered_detections)

    cv2.polylines(frame, [polygon_points.reshape((-1, 1, 2))], isClosed=True, color=(0, 0, 255), thickness=2)

    for track in tracker.tracked_plates:
        box = track['bbox']
        track_id = track['id']

        x1, y1, x2, y2 = map(int, box)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Hiển thị frame đã xử lý
    cv2.imshow('License Plate Tracking', frame)

    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


d:\work\AIVISION\license-plate\license-plate\license-plate\License-Plate-Recognition\yolov5\models\common.py:565: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
d:\work\AIVISION\license-plate\license-plate\license-plate\License-Plate-Recognition\yolov5\models\common.py:565: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
d:\work\AIVISION\license-plate\license-plate\license-plate\License-Plate-Recognition\yolov5\models\common.py:565: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
d:\work\AIVISION\license-plate\license-plate\license-plate\License-Plate-Recognition\yolov5\models\common.py:565: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch

In [2]:
import cv2
import numpy as np

# Danh sách lưu các điểm đã click
points = []

# Hàm xử lý sự kiện chuột
def click_event(event, x, y, flags, param):
    global points, frame
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        # Vẽ một vòng tròn nhỏ tại điểm click để đánh dấu
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow('Frame', frame)
        print(f"Đã chọn điểm: ({x}, {y})")

# Mở video từ file hoặc webcam
# Nếu dùng file video, thay 'duongdan/video.mp4' bằng đường dẫn đến file của bạn
# Nếu muốn dùng webcam, thay cv2.VideoCapture(0)
cap = cv2.VideoCapture('./test_image/test3.mp4')

if not cap.isOpened():
    print("Không thể mở video!")
    exit()

# Đọc frame đầu tiên của video để chọn tọa độ
ret, frame = cap.read()
if not ret:
    print("Không thể đọc frame từ video!")
    exit()

# Hiển thị frame và thiết lập callback cho sự kiện chuột
cv2.imshow('Frame', frame)
cv2.setMouseCallback('Frame', click_event)
print("Vui lòng click chọn 4 điểm trên frame. Nhấn ESC để kết thúc.")

# Chờ người dùng click đủ 4 điểm hoặc nhấn ESC
while True:
    key = cv2.waitKey(1) & 0xFF
    if key == 27 or len(points) == 4:  # 27 là mã của ESC
        break

cv2.destroyWindow('Frame')

# Kiểm tra nếu đã chọn đủ 4 điểm
if len(points) != 4:
    print("Bạn chưa chọn đủ 4 điểm!")
    cap.release()
    exit()

print("Các tọa độ đã chọn:", points)

# Chuyển danh sách điểm sang mảng NumPy và định dạng lại để vẽ đa giác
pts = np.array(points, np.int32)
pts = pts.reshape((-1, 1, 2))

# Tiếp tục xử lý video: vẽ đa giác lên từng frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Vẽ đa giác (nối liền các điểm đã chọn)
    cv2.polylines(frame, [pts], isClosed=True, color=(255, 0, 0), thickness=2)

    cv2.imshow('Video', frame)

    # Nhấn ESC để thoát video
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Vui lòng click chọn 4 điểm trên frame. Nhấn ESC để kết thúc.
Đã chọn điểm: (89, 607)
Đã chọn điểm: (331, 344)
Đã chọn điểm: (1296, 352)
Đã chọn điểm: (1313, 641)
Các tọa độ đã chọn: [(89, 607), (331, 344), (1296, 352), (1313, 641)]


In [6]:
import cv2
import numpy as np
import torch
import collections
import function.helper as helper
import function.utils_rotate as utils_rotate

# -----------------------------------------
# 1. Khởi tạo model và các tham số cần dùng
# -----------------------------------------

# Khởi tạo model detection biển số (model này sẽ trả về các bbox biển số)
yolo_LP_detect = torch.hub.load('yolov5', 'custom', path='model/LP_detector.pt', force_reload=True, source='local')

# Khởi tạo model OCR biển số
yolo_license_plate = torch.hub.load('yolov5', 'custom', path='model/LP_ocr.pt', force_reload=True, source='local')

# Định nghĩa vùng đa giác (chỉ xử lý đối tượng khi bbox nằm trong vùng này)
polygon_points = np.array([(89, 607), (331, 344), (1296, 352), (1313, 641)], np.int32)

# Mở video (hoặc dùng webcam bằng cách thay './test_image/test3.mp4' thành 0)
cap = cv2.VideoCapture('./test_image/test3.mp4')

# Giả sử bạn đã định nghĩa tracker (ví dụ: LicensePlateTracker)
# Tracker này sẽ quản lý các đối tượng được phát hiện qua nhiều frame.
tracker = LicensePlateTracker()

# -----------------------------------------
# 2. Vòng lặp xử lý từng frame
# -----------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- Phát hiện biển số trên frame với yolo_LP_detect ---
    results = yolo_LP_detect(frame, size=640)
    # Lấy danh sách detection với định dạng [x1, y1, x2, y2, conf, class, ...]
    detections = results.pandas().xyxy[0].values.tolist()

    # --- Lọc detection: chỉ giữ các bbox mà tâm nằm trong vùng đa giác ---
    filtered_detections = []
    for det in detections:
        x1, y1, x2, y2 = map(int, det[:4])
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Nếu tâm bbox nằm trong đa giác (>=0: bên trong hoặc trên biên)
        if cv2.pointPolygonTest(polygon_points, (center_x, center_y), False) >= 0:
            filtered_detections.append(det)

    # --- Cập nhật tracker với các detection đã lọc ---
    tracker.update(filtered_detections)

    # --- Khai báo list lưu kết quả OCR của từng đối tượng trong frame hiện tại ---
    ocr_results = []

    # --- Duyệt qua các đối tượng được tracker theo dõi ---
    for track in tracker.tracked_plates:
        bbox = track['bbox']
        track_id = track['id']

        # Ép kiểu tọa độ thành số nguyên
        x1, y1, x2, y2 = map(int, bbox)
        # Vẽ bbox và ID trên frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # --- Cắt ROI (vùng biển số) từ frame ---
        roi = frame[y1:y2, x1:x2]
        if roi.size == 0:
            continue

        # --- Thực hiện OCR trên ROI ---
        # Sử dụng hàm read_plate() trong helper để nhận diện ký tự từ biển số
        lp_result = helper.read_plate(yolo_license_plate, roi)

        # Nếu kết quả là "unknown", thử deskew (hiệu chỉnh góc) theo 2 giá trị khác nhau
        if lp_result == "unknown":
            flag = False
            for cc in range(0, 2):
                for ct in range(0, 2):
                    lp_result = helper.read_plate(yolo_license_plate, utils_rotate.deskew(roi, cc, ct))
                    if lp_result != "unknown":
                        flag = True
                        break
                if flag:
                    break

        # Nếu đã nhận diện được biển số, lưu kết quả và vẽ kết quả lên frame
        if lp_result != "unknown":
            ocr_results.append(lp_result)
            cv2.putText(frame, lp_result, (x1, y2 + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # --- Thống kê và hiển thị kết quả OCR xuất hiện nhiều nhất trong frame ---
    if ocr_results:
        most_common, count = collections.Counter(ocr_results).most_common(1)[0]
        cv2.putText(frame, f"ID: {track_id} LP:{most_common}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 216, 255), 2)

    # --- Vẽ vùng đa giác lên frame để trực quan ---
    cv2.polylines(frame, [polygon_points.reshape((-1, 1, 2))], isClosed=True, color=(0, 0, 255), thickness=2)

    # Hiển thị frame đã xử lý
    cv2.imshow("Tracking and OCR", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()


YOLOv5  v6.1-179-gf3fecf9 torch 2.5.1 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

d:\work\AIVISION\license-plate\license-plate\license-plate\License-Plate-Recognition\yolov5\models\experimental.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open a

In [15]:
pwd

'd:\\work\\AIVISION\\license-plate\\license-plate\\license-plate\\License-Plate-Recognition'